<b>Personal Info:</b><br>
<b>Name:</b> Ehud Michel<br>
<b>ID:</b> 208952713


<b>Importing the packages:

In [1]:
# imports for reading and writing (input & output) files:
import pandas as pd
import os
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import hebrew_tokenizer as hz
from operator import itemgetter

/Users/ehudmichel/Documents/studies/machine_learning/pythonProjectMatala


/Users/ehudmichel/opt/anaconda3/lib/python3.8/site-packages/hebrew_tokenizer/tokenizer.py:121: FutureWarning: Possible nested set at position 729
  self.scanner = re.compile(


<b>Getting the data:

In [2]:
train_filename = '.' + os.sep + 'input' + os.sep + 'annotated_corpus_for_train.xlsx'
test_filename  = '.' + os.sep + 'input' + os.sep + 'corpus_for_test.xlsx'
df_train = pd.read_excel(train_filename, 'corpus', index_col=None, na_values=['NA'])
df_test  = pd.read_excel(test_filename,  'corpus', index_col=None, na_values=['NA'])

<b>function that uses the hebrew tokenizer and generates:<br> 
    1.list with the counts of the group types for each story - "num_grg_aper"<br>
    2.list with the most common tokens with thier counts of apperences - "most countes_words"<br>
    3.set with all the group types - "token_set":<br> 
        ('HEBREW','PUNC','NUMBER',etc..)<br>
    4.set with all the common types - "all_words_set"<br>

<b> function to inzilize values and new features to the manual tokenization:

In [ ]:
def get_text_idexes_test(story_tokens,num_of_types_per_story):   
    tokens_set = set()
    all_words_set = set()
    num_grg_aper = []
    most_countes_words = []
    for i in range(len(story_tokens)):
        number_group_occur_dict = {}
        number_words_occur_dict = {}
        for group,token,token_num,(s_i,e_i) in story_tokens[i]:
            if group in list(number_group_occur_dict.keys()):
                number_group_occur_dict[group] += 1
            else:
                number_group_occur_dict[group] = 1
            if group =='HEBREW':
                if token in list(number_words_occur_dict.keys()):
                    number_words_occur_dict[token] +=1
                else:
                    number_words_occur_dict[token] = 1
        top_words_dict = dict(sorted(number_words_occur_dict.items(),key=itemgetter(1),reverse=True)[:num_of_types_per_story])
        tokens_set.update(list(number_group_occur_dict.keys()))
        all_words_set.update(list(top_words_dict.keys()))
        num_grg_aper.append(number_group_occur_dict)
        most_countes_words.append(top_words_dict)
    return num_grg_aper, tokens_set, most_countes_words, all_words_set

In [ ]:
def inizilaing_vals_in_df_manual_test(df_train,num_of_types_per_story):
    lang = ['HEBREW','ENGLISH']
    df_train_copy = df_test.copy() # generating copy of the dataset
    df_train_copy['tokenizer'] = df_train_copy.story.apply(lambda x: hz.tokenize(x,with_whitespaces=True)) #adding a 
    # tokeniazer column that contain tokenize object related to to story
    
    df_train_copy['len'] = df_train_copy['story'].apply(lambda x: len(x))#adding column name len that contains the length of each story
    num_group_count_for_story, token_type_set, most_common_words, all_words = get_text_idexes_test(df_train_copy['tokenizer'],num_of_types_per_story)
    #num_group_count_for_story - list of all the dictionaries that each contains counts of the toknizer groups apearences
    #token_type_set            - set with all the possible token type ('HEBREW','PANCTUAION','DATE','NUMBER','WHITE_SPACE')
    #most_common_words         - list that contains dictionary for each story with its nth(defiend in the summery function) number of most common words
    #all_words                 - set with most common words from all the stories togather

    for token in token_type_set:
        df_train_copy.insert(loc=0,column=token,value=0) #inizilizng every cell in the tokens types colmuns with 0's
    for word in all_words:
        df_train_copy.insert(loc=len(token_type_set)+2,column=word,value=0) #adding column for each word in the all words set and inizilizng every cell in every word column with 0's
    for i in range(df_train_copy.shape[0]): # for each row in the data
        for group_type in list(num_group_count_for_story[i].keys()):# for each group type in the ith story
            df_train_copy.loc[i, group_type] = num_group_count_for_story[i][group_type]
        for type_ in list(most_common_words[i].keys()): # for each type in the list[i] of dictionary keys of most common types
            df_train_copy.loc[i, type_] = most_common_words[i][type_]
    df_train_copy = df_train_copy.drop(['story','tokenizer'],axis=1)
    df_train_copy.insert(loc=len(token_type_set),column='total',value=df_train_copy[token_type_set].sum(axis=1))
    df_train_copy.insert(loc=len(token_type_set),column='amount_of_words',value=df_train_copy[lang].sum(axis=1))
    df_train_copy.insert(loc=len(token_type_set),column='len_of_paragraph',value=df_train_copy['story'].apply(lambda x: len(x)))
    df_train_copy.insert(loc=len(token_type_set),column='len_of_paragraph_per_word',value=df_train_copy['len_of_paragraph'] / df_train_copy['amount_of_words'])
    df_train_copy.insert(loc=len(token_type_set),column='Len',value=df_train_copy['len'])
    
    df_train_copy.insert(loc=len(token_type_set),column='len_of_words',value=df_train_copy['Len'] / df_train_copy['amount_of_words'])

    # adding amount of words in story column for each story
    df_train_copy['len_of_words'] = df_train_copy['len'] / df_train_copy['amount_of_words'] # adding column with the number of tokens divide by number of types
    del df_train_copy['len']
    df_stats = df_train_copy.iloc[:,:13]
    df_manual = df_train_copy.iloc[:,14:]
    return df_stats,token_type_set,df_manual,all_words,

<b>Function to print confusion matrix and classification report:

In [ ]:
def print_conMatrix(y_test,y_pred):
    print(f'Confusion Matrix:\n {confusion_matrix(y_test,y_pred)}')
      
    print (f'Accuracy:\n {round(accuracy_score(y_test,y_pred)*100, 4)}%')

    print(f'Report:\n {classification_report(y_test,y_pred)}')

<b>Function that sums the number of types occurnces in df:

In [ ]:
def type_sum(df_01):
    df = df_01.copy()
    dict_  = {}
    for col in df.columns:
        dict_[col] = df[col].sum()
    dict_sorted = sorted(dict_.items(), key=operator.itemgetter(1),reverse=True)
    return dict_sorted

In [ ]:
df_stats,stats_colmuns,df_manual,all_wrds_set = inizilaing_vals_in_df_manual(df_train,175)
# עיבוד מידע על הדאטה בצורה ידנית

In [ ]:
dict_df_manual = type_sum(df_manual)

df_manual_droped = df_manual.copy()
col_to_drop = []
for i,j in dict_df_manual:
    if(j <=10):
        col_to_drop.append(i)
df_manual_droped = df_manual_droped.drop(columns=col_to_drop)
df_manual_droped = df_manual_droped.copy()
col_to_drop = []
for i,j in dict_df_manual:
    if(j >=58):
        col_to_drop.append(i)
df_manual_droped = df_manual_droped.drop(columns=col_to_drop)
df_merge = pd.concat([df_stats,df_manual_droped],axis=1)
X = df_merge
y = df_train.gender
y = y.apply(lambda x:0 if x=='m' else 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
clf_LG = LogisticRegression(random_state=0,max_iter=10000000).fit(X_train, y_train)
y_pred_LG = clf_LG.predict(X_test)
print_conMatrix(y_test=y_test,y_pred=y_pred_LG)

In [ ]:
clf_LG = LogisticRegression(random_state=0,max_iter=10000000).fit(X, y)
y_pred_LG = clf_LG.predict(X_test)
print_conMatrix(y_test=y_test,y_pred=y_pred_LG)

In [3]:
all_dfs = pd.concat([df_train,df_test])
all_dfs

,story,gender,test_example_id
0,"בוקר אחד קמתי סהרורי יצאתי מהמיטה קצת מטושטש ,...",m,NaN
1,לחבר שלי היה יום הולדת וחיפשנו מה אפשר לעשות ל...,m,NaN
2,"השנה האחרונה הייתה שנת קורונה, שנה לא פשוטה בק...",m,NaN
3,"לפני כחצי שנה עברתי לגור בצפון עם בת זוגתי, עב...",m,NaN
4,"יום חמישי רגיל, תמיד מתחיל לעבור טיפה מאוחר יו...",m,NaN
...,...,...,...
151,לא הרבה יודעים אך אני מתעסק הרבה בשוק ההון ובמ...,NaN,151.0
152,"כשהייתי בן 4 נולד לי אח קטן, אני חייב להודות ש...",NaN,152.0
153,"הכל התחיל אי שם בגיל ההתבגרות, היתה לי חברה מו...",NaN,153.0
154,"נסעתי עם חברים לים, היה יום חם ורצינו ללכת לנצ...",NaN,154.0


In [22]:
def get_text_idexes(story_tokens,num_of_types_per_story):
    
    tokens_set = set()
    all_words_set = set()
    num_grg_aper = []
    most_countes_words = []
    for i in range(len(story_tokens)):
        number_group_occur_dict = {}
        number_words_occur_dict = {}
        for group,token,token_num,(s_i,e_i) in story_tokens[i]:
            if group in list(number_group_occur_dict.keys()):
                number_group_occur_dict[group] += 1
            else:
                number_group_occur_dict[group] = 1
            if group =='HEBREW':
                if token in list(number_words_occur_dict.keys()):
                    number_words_occur_dict[token] +=1
                else:
                    number_words_occur_dict[token] = 1
        top_words_dict = dict(sorted(number_words_occur_dict.items(),key=itemgetter(1),reverse=True)[:num_of_types_per_story])
        tokens_set.update(list(number_group_occur_dict.keys()))
        all_words_set.update(list(top_words_dict.keys()))
        num_grg_aper.append(number_group_occur_dict)
        most_countes_words.append(top_words_dict)
    return num_grg_aper, tokens_set, most_countes_words, all_words_set

In [23]:
def inizilaing_vals_in_df_manual(all_dfs,num_of_types_per_story):
    lang = ['HEBREW','ENGLISH']
    df_train_copy = all_dfs.copy() # generating copy of the dataset
    df_train_copy['tokenizer'] = df_train_copy.story.apply(lambda x: hz.tokenize(x,with_whitespaces=True)) #adding a 
    # tokeniazer column that contain tokenize object related to to story
    
    df_train_copy.gender = df_train_copy.gender.apply(lambda x:0 if x=='m' else 1) # modying m->0,f->1
    df_train_copy['len'] = df_train_copy['story'].apply(lambda x: len(x))#adding column name len that contains the length of each story
   
    num_group_count_for_story, token_type_set, most_common_words, all_words = get_text_idexes(df_train_copy['tokenizer'],num_of_types_per_story)
   
    #num_group_count_for_story - list of all the dictionaries that each contains counts of the toknizer groups apearences
    #token_type_set            - set with all the possible token type ('HEBREW','PANCTUAION','DATE','NUMBER','WHITE_SPACE')
    #most_common_words         - list that contains dictionary for each story with its nth(defiend in the summery function) number of most common words
    #all_words                 - set with most common words from all the stories togather

    for token in token_type_set:
        df_train_copy.insert(loc=0,column=token,value=0) #inizilizng every cell in the tokens types colmuns with 0's
    for word in all_words:
        df_train_copy.insert(loc=len(token_type_set)+2,column=word,value=0) #adding column for each word in the all words set and inizilizng every cell in every word column with 0's
    for i in range(df_train_copy.shape[0]): # for each row in the data
        for group_type in list(num_group_count_for_story[i].keys()):# for each group type in the ith story
            df_train_copy.loc[i, group_type] = num_group_count_for_story[i][group_type]
        for type_ in list(most_common_words[i].keys()): # for each type in the list[i] of dictionary keys of most common types
            df_train_copy.loc[i, type_] = most_common_words[i][type_]
    df_train_copy = df_train_copy.drop(['story','tokenizer'],axis=1)
    df_train_copy.insert(loc=len(token_type_set),column='total',value=df_train_copy[token_type_set].sum(axis=1))
    df_train_copy.insert(loc=len(token_type_set),column='amount_of_words',value=df_train_copy[lang].sum(axis=1))
    df_train_copy.insert(loc=len(token_type_set),column='len_of_paragraph',value=df_train['story'].apply(lambda x: len(x)))
    df_train_copy.insert(loc=len(token_type_set),column='len_of_paragraph_per_word',value=df_train_copy['len_of_paragraph'] / df_train_copy['amount_of_words'])
    df_train_copy.insert(loc=len(token_type_set),column='Len',value=df_train_copy['len'])
    
    df_train_copy.insert(loc=len(token_type_set),column='len_of_words',value=df_train_copy['Len'] / df_train_copy['amount_of_words'])

    # adding amount of words in story column for each story
    df_train_copy['len_of_words'] = df_train_copy['len'] / df_train_copy['amount_of_words'] # adding column with the number of tokens divide by number of types
    del df_train_copy['len']
    df_stats = df_train_copy.iloc[:,:13]
    df_manual = df_train_copy.iloc[:,14:]
    return df_stats,token_type_set,df_manual,all_words,

In [ ]:
dict_df_manual_test = type_sum(df_manual_test)

df_manual_droped_test = df_manual_test.copy()
col_to_drop = []
for i,j in dict_df_manual_test:
    if(j <=10):
        col_to_drop.append(i)
df_manual_droped_test = df_manual_droped_test.drop(columns=col_to_drop)
df_manual_droped_test = df_manual_droped_test.copy()
col_to_drop = []
for i,j in dict_df_manual_test:
    if(j >=58):
        col_to_drop.append(i)
df_manual_droped_test = df_manual_droped_test.drop(columns=col_to_drop)
df_merge_test = pd.concat([df_stats_test,df_manual_droped_test],axis=1)
X_test_real = df_merge_test
predicted_category = clf_LG.predict(X_test_real)


In [ ]:
X_test_real

In [ ]:
df_stats_test,stats_colmuns_test,df_manual_test,all_wrds_set_test = inizilaing_vals_in_df_manual_test(df_test,175)

In [ ]:
len(predicted_category)

In [ ]:
predicted_category = pd.Series(predicted_category)

In [ ]:
predicted_category = predicted_category.apply(lambda x:'m' if x==0 else 'f')


In [ ]:
test_example_id = df_test['test_example_id']

In [25]:
df_stats_test,stats_colmuns_test,df_manual_test,all_wrds_set_test = inizilaing_vals_in_df_manual(all_dfs,175)


520


ValueError: too many values to unpack (expected 4)

In [ ]:
data = {'test_example_id':test_example_id,
       'predicted_category': predicted_category}

In [ ]:
classification_results = pd.DataFrame(data=data)

In [ ]:
classification_results

In [ ]:
df_stats_all,stats_colmuns_all,df_manual_all,all_wrds_set_all = inizilaing_vals_in_df_manual(all_dfs,130)